### This notebook registers the model stored in output/ and generates the necessary scoring scripts for deploying the model to the Azure machine learning workspace using Azure container Instance.

In [20]:
import torch
import torch.nn as nn
from torchvision import transforms
import json
import pretrainedmodels
import os,sys
import numpy as np
import azureml.core
from azureml.core.image import ContainerImage
from azureml.core.image import Image
import azureml.data
from azureml.data.data_reference import DataReference
from azureml.core import Workspace, Datastore
from azureml.core import Run
import os, uuid, sys
from azure.storage.blob import BlockBlobService, PublicAccess
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
import shutil
model_name = 'nasnetalarge' # could be fbresnet152 or inceptionresnetv2
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='')
for param in model.parameters():
    param.requires_grad = False
ws = Workspace.from_config()


In [4]:
# setting up all the base paths.
base_path = os.path.dirname(os.getcwd())
data_raw_dir = os.path.join(base_path,'data/raw')
data_proc_dir = os.path.join(base_path,'data/processed')
data_img_dir = os.path.join(base_path,'data/processed/images')
label_file = os.path.join(data_raw_dir,'Data_Entry_2017.csv')
notshared_dir=os.path.join(base_path,'notshared')
model_path = os.path.join(base_path,'output/full_chexray_nasnet_e17.pth.tar')
DISEASE_LIST = ['Atelectasis','Cardiomegaly','Consolidation','Edema','Effusion','Emphysema','Fibrosis','Hernia','Infiltration','Mass','Nodule','Pleural_Thickening','Pneumonia','Pneumothorax']

In [48]:
model_name = 'nasnetalarge' # could be fbresnet152 or inceptionresnetv2
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='')

for param in model.parameters():
    param.requires_grad = False

In [49]:
n_classes=14

in_ftrs=model.last_linear.in_features
model.last_linear=nn.Sequential(
nn.Linear(in_ftrs,256),
    nn.ReLU(),
    nn.Dropout(p=0.4),
    nn.Linear(256,n_classes),
nn.Sigmoid())


In [ ]:
chkpt = torch.load(model_path, map_location=lambda storage, loc: storage)
state_dict = chkpt['state_dict']


from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove 'module.' of dataparallel
    new_state_dict[name]=v

model.load_state_dict(new_state_dict)

In [ ]:
#try out the model on a sample before deploying.

import pretrainedmodels.utils as utils

load_img = utils.LoadImage()

# transformations depending on the model
# rescale, center crop, normalize, and others (ex: ToBGR, ToRange255)
tf_img = utils.TransformImage(model) 

path_img = os.path.join(data_img_dir,'00000044_001.png')
input_img = load_img(path_img)
input_tensor = tf_img(input_img) 
input_tensor.shape
input_tensor = input_tensor.unsqueeze(0) 
input = torch.autograd.Variable(input_tensor,
    requires_grad=False)

output = model(input) 
print(output)
vals,ids = output.topk(5)
print(ids.tolist()[0])
print(vals)
for idx in ids.tolist()[0]:
    print(DISEASE_LIST[idx])


In [56]:
import shutil
shutil.copy('../output/full_chexray_nasnet_e17.pth.tar','./')

'./full_chexray_nasnet_e17.pth.tar'

In [55]:
model = Model.register(workspace = ws,
                       model_path ='full_chexray_nasnet_e17.pth.tar',
                       model_name = "full_chexray_e17",
                       tags = {"chest xray": "demo"},
                       description = "chest xray scan classification")

Registering model full_chexray_e17


In [57]:
with open(os.path.join(notshared_dir,'credentials.json')) as creds:    
    credentials = json.load(creds)

In [16]:
%%writefile ../src/pytorch_run.py

import torch
import torch.nn as nn
from torchvision import transforms
import json
import pretrainedmodels
import os, uuid, sys
from PIL import Image
from io import BytesIO
from azure.storage.blob import BlockBlobService, PublicAccess
import pretrainedmodels.utils as utils
from collections import OrderedDict
import base64
import logging
import time


DISEASE_LIST = ['Atelectasis','Cardiomegaly','Consolidation','Edema','Effusion','Emphysema','Fibrosis','Hernia','Infiltration','Mass','Nodule','Pleural_Thickening','Pneumonia','Pneumothorax']

from azureml.core.model import Model
global img_string
img_string = ''

# helper functions
def im_2_b64(image):
    buff = BytesIO()
    image.save(buff, format="PNG")
    img_str = base64.b64encode(buff.getvalue())
    return img_str

def b64_2_img(data):
    buff = BytesIO(base64.b64decode(data))
    buff.seek(0)
    return Image.open(buff)

def init():
    global model
    seed = 0
    torch.manual_seed(seed)
    logging.basicConfig(level=logging.DEBUG)
    model_name = 'nasnetalarge'
    model = pretrainedmodels.__dict__[model_name](num_classes=1000,pretrained='')
    model_path = Model.get_model_path('full_chexray_e17')
    
    # if you want to test the run script on your local system you can uncomment the following line and comment the one above
    #model_path = os.path.join(base_path,'output/chexray_nasnet_e10.pth.tar')
    n_classes=14
    
    in_ftrs=model.last_linear.in_features
    model.last_linear=nn.Sequential(
    nn.Linear(in_ftrs,256),
    nn.ReLU(),
    nn.Dropout(p=0.4),
    nn.Linear(256,n_classes),
    nn.Sigmoid())
    
    chkpt = torch.load(model_path, map_location=lambda storage, loc: storage)
    state_dict = chkpt['state_dict']
    
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] # remove 'module.' of dataparallel
        new_state_dict[name]=v

    model.load_state_dict(new_state_dict)
    global load_img
    load_img = utils.LoadImage()
    global credentials
    credentials = {
        "datastore_name" : "chexrayds",
        "container_name" : "chexray-images",
        "account_name" : "chexray14",
        "account_key" : "rXogWjkm8lXV0QfHAety1KktKGxbb1pMGoWkd4E9FI3QN+MW2qp4bWlFRymGS2zR0MyroV+VJcgHurr8+dhQdg=="
    } 
    model.eval()


def run(input_data):
    #print(input_data)
    imgstring = json.loads(input_data)['data']
    img_name = json.loads(input_data)['file_name']
    should_upload = json.loads(input_data)['upload']
    try:
        print(img_name)
        IMAGENET_RGB_MEAN = [0.485, 0.456, 0.406]
        IMAGENET_RGB_SD = [0.229, 0.224, 0.225]

        normalize = transforms.Normalize(IMAGENET_RGB_MEAN, IMAGENET_RGB_SD)

        # the following transformation is applied on each incoming message.
        transform=transforms.Compose([
                                 transforms.Resize((331,331),interpolation=Image.NEAREST),
                                 transforms.ToTensor(),  
                                 normalize])

        img = b64_2_img(imgstring)
        
        # upload the image to blob storage, if sufficient permission is given.
        if should_upload == True:
            base_path = os.getcwd()
            

            timestr=time.strftime("%Y%m%d-%H%M%S")
            file_name = img_name + timestr

            full_path_to_file = os.path.join(base_path, file_name)
            img.save(file_name +'.png','PNG')
            block_blob_service = BlockBlobService(account_name = credentials['account_name'], account_key = credentials['account_key']) 
            block_blob_service.create_blob_from_path(credentials['container_name'], 'images/'+file_name+'.png', full_path_to_file+'.png')
            print('uploaded '+ file_name+'.png')
            os.remove(file_name+'.png')
            
        # get prediction
        with torch.no_grad():
            input_tensor = transform(img.convert('RGB'))
            input_tensor = input_tensor.unsqueeze(0)
            print(input_tensor.shape)
            output = model(input_tensor)
            vals,ids = output.topk(5)
            data = {}
            labels = []
            for id in ids.tolist()[0]:
                labels.append(DISEASE_LIST[id])
            print(labels)
            data['labels'] = labels;
            print(vals.tolist()[0])
            data['probabilities'] = vals.tolist()[0]
            total_sum = 0
            for val in vals.tolist()[0]:
                total_sum = total_sum + val
            print(total_sum)



        result = {
            "labels": labels,
            "probabilities": vals.tolist()[0],
            "comments": "",
            "prob_sum": total_sum
        }
    
    except Exception as e:
        print("Unexpected error:", str(e))
        result = {
            "comments": str(e), 
        }
        return result
    
    
    print(result)
    return result


Overwriting ../src/pytorch_run.py


In [ ]:
# to generate the conda dependencies file.
!conda env export --no-builds > ../env.yml

In [17]:
# scoring file must be in the same directory as notebook for the deployment, so we copy it temporarily.
shutil.copy('../src/pytorch_run.py','./')



image_config = ContainerImage.image_configuration(runtime="python",
                                                  execution_script="pytorch_run.py",
                                                  conda_file="../env.yml")

In [18]:
ml =Model.list(ws,name='full_chexray_e17')
model= ml[0]

In [21]:

image = Image.create(name='myimg', models=[ml[0]], image_config=image_config, workspace=ws)

# wait for image creation to finish
image.wait_for_creation(show_output=True)

Creating image
Running.............................................................................................................................................................................................
Succeeded
Image creation operation finished for image myimg:31, operation "Succeeded"


In [ ]:


aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4)

aci_service = Webservice.deploy_from_image(deployment_config=aci_config, 
                                           image=image, 
                                           name='svc', 
                                           workspace=ws)
aci_service.wait_for_deployment(show_output=True) 


In [5]:
print(ws.webservices['svc'].get_logs())

...","upload":true,"file_name":"00000011_000.png"}
Headers passed in (total 11):
	Host: localhost:5001
	X-Real-Ip: 127.0.0.1
	X-Forwarded-For: 127.0.0.1
	X-Forwarded-Proto: http
	Connection: close
	Content-Length: 605196
	User-Agent: Go-http-client/1.1
	Accept: application/json
	Content-Type: application/json
	X-Ms-Request-Id: ad22a66b-c6d5-459c-ab00-133380256fdf
	Accept-Encoding: gzip
Scoring Timer is set to 3600.0 seconds
00000011_000.png
torch.Size([3, 331, 331])
torch.Size([1, 3, 331, 331])
post o/p
['Infiltration', 'Effusion', 'Atelectasis', 'Pleural_Thickening', 'Mass']
[0.11818211525678635, 0.08885078132152557, 0.08108849078416824, 0.03915303573012352, 0.037541039288043976]
0.36481546238064766
{'labels': ['Infiltration', 'Effusion', 'Atelectasis', 'Pleural_Thickening', 'Mass'], 'probabilities': [0.11818211525678635, 0.08885078132152557, 0.08108849078416824, 0.03915303573012352, 0.037541039288043976], 'comments': '', 'prob_sum': 0.36481546238064766}
200
127.0.0.1 - - [27/Jun/2019

In [18]:
print(aci_service.state)

Healthy
